In [ ]:
import tensorflow as tf

In [ ]:
tf.reset_default_graph()

In [ ]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("data", one_hot=True)

In [ ]:
def weight(shape):
    ini = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(ini)

def bias(shape):
    ini = tf.constant(0.1, shape = shape)
    return tf.Variable(ini)

def conv_layer(inpu, shape, strides=[1,1,1,1]):
    W = weight(shape)
    b = bias([shape[3]])
    return tf.nn.relu(tf.nn.conv2d(inpu, W, strides=strides, padding="SAME")+b)

def fully_connected(inpu, outpu):
    W = weight([int(inpu.get_shape()[1]), outpu])
    b = bias([outpu])
    return tf.matmul(inpu, W) + b

In [ ]:
X = tf.placeholder(tf.float32, shape=[None, 784])
x_image = tf.reshape(X, [-1,28,28,1])
y = tf.placeholder(tf.float32, shape=[None,10])
keep_prob = tf.placeholder(tf.float32)

In [ ]:
#9 convolution layers, 3 pool layers, 4 dropouts

# the first block of convolution layers
conv1_1 = conv_layer(X, shape=[3,3,1,32])
conv1_2 = conv_layer(conv1_1, shape=[3,3,32,32])
conv1_3 = conv_layer(conv1_2, shape=[3,3,32,32])
conv1_pool = tf.nn.max_pool(conv1_3, ksize=[1,2,2,1], strides=[1,2,2,1], padding="SAME")
conv1_drop = tf.nn.dropout(conv1_pool, keep_prob=keep_prob)

# the second block of convolution layers
conv2_1 = conv_layer(conv1_drop, shape=[3,3,32,64])
conv2_2 = conv_layer(conv2_1, shape=[3,3,64,64])
conv2_3 = conv_layer(conv2_2, shape=[3,3,64,64])
conv2_pool = tf.nn.max_pool(conv2_3, ksize=[1,2,2,1], strides=[1,2,2,1], padding="SAME")
conv2_drop = tf.nn.dropout(conv2_pool, keep_prob=keep_prob)

# the third block of convolution layers
conv3_1 = conv_layer(conv2_drop, shape=[3,3,64,128])
conv3_2 = conv_layer(conv3_1, shape=[3,3,128,128])
conv3_3 = conv_layer(conv3_2, shape=[3,3,128,128])
conv3_pool = tf.nn.max_pool(conv3_3, ksize=[1,2,2,1], strides=[1,2,2,1], padding="SAME")
conv3_drop = tf.nn.dropout(conv3_pool, keep_prob=keep_prob)

#ready for fully_connected
conv3_flat = tf.reshape(conv3_pool, [-1, 4*4*128])
conv3_drop = tf.nn.dropout(conv3_flat, keep_prob = keep_prob)
full1 = tf.nn.relu(fully_connected(conv3_flat, 512))
full1_drop = tf.nn.dropout(full1,keep_prob=keep_prob)

# And finally the logits
y_conv = fully_connected(full1_drop, 10)

In [ ]:
xentropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=y_conv, labels=y))
train = tf.train.AdamOptimizer(1e-4).minimize(xentropy)

In [ ]:
correct = tf.equal(tf.argmax(y_conv, 1), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))

In [ ]:
steps = 200000 #It takes many steps to converge
batch_size = 50

final_accuracy = 0
saver = tf.train.Saver()

In [ ]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    for i in range(steps):
        batch = mnist.train.next_batch(batch_size)
        sess.run(train, feed_dict={X: batch[0], y: batch[1], keep_prob: 0.5})
        
        if (i+1) % 5000 == 0:
            train_accuracy = sess.run(accuracy, feed_dict={X: batch[0], y: batch[1], keep_prob: 1.0})
            test_accuracy = sess.run(accuracy, feed_dict={X: mnist.test.images, y: mnist.test.labels, keep_prob: 1.0})
            print("Step {}, \tTraining accuracy {:.4f}, \tTest accuarcy {:.4f}".format((i+1), train_accuracy, test_accuracy))
            if test_accuracy > final_accuracy:
                final_accuracy = test_accuracy
                save_model = saver.save(sess, "models/CNN_MNIST_final.ckpt")
    print("\nFinal Accuracy {}".format(final_accuracy))